<a href="https://colab.research.google.com/github/alvarofpinheiro/pifwia_pso/blob/main/PIFWIA_PSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PSO - Particle Swarm Optimization ou Otimização por Enxame de Partículas uma metaheurística proposta pelos pesquisadores Kennedy e Eberhart em 1995 para resolver problemas de otimização inspirado no comportamento das particulas, buscando melhorar a solução candidata com aplicação de uma dada medida de qualidade.

Kennedy, J.; Eberhart, R.  Particle swarm optimization. IEEE Xplore. https://doi.org/10.1109/ICNN.1995.488968, 1995.

In [ ]:
#instala biblioteca Orange Canvas
!pip install Orange3

In [ ]:
#importa bibliotecas
import Orange
import random
import numpy as np
from numpy.random import choice
import matplotlib.pyplot as plt
from sklearn import metrics

In [ ]:
#define os hiperparâmetros
DIMENSOES = 2 #determina a quantidade de dimensões do problema
ITERACOES = 200 #quantiddade máxima de ciclos defindo quantas explorações serão feitas (episódios)
POPULACAO = 20 #tamanho da população correspondente ao número de partículas na espaço de busca
E1 = -15 #extremo esquerdo eixo x
E2 = 15 #extremo direito eixo x
E3 = -100 #extremo inferior eixo y
E4 = 100 #extremo superior eixo y
LIMITES = [E3,E4] #(bound) determina os valores maximos e minimos do espaço de busca (região de busca)
FCUSTO = 'rosenbrock' #(fitness) função custo que pode ser uma esfera ou parábola (rosenbrock) ou outra função custo que defina a densidade de partículas na região de busca
CI = 0.8 #(𝑤) coeficiente de inercia que determina o quanto a velocidade anterior influencia na velocidade atual
FI = 2.05 #(𝑐1) fator de individualidade que influencia na atração que a partícula tem em direção à melhor posição já encontrada por ela mesma (𝑃𝑏𝑒𝑠𝑡)
FS = 2.05 #(𝑐2) fator de sociabilidade que influencia na atração que a partícula tem em direção à melhor posição já encontrada por qualquer partícula vizinha a ela (𝐿𝑏𝑒𝑠𝑡)
VM = [-5,5] #velocidade máxima
PARTICULAS = [] #(swarm) array da criação das particulas

In [ ]:
#importa dados
from google.colab import files  
files.upload()

In [ ]:
#instancia objeto de dados com base no caminho gerado na importação do arquivo
dados = Orange.data.Table("/content/dados.csv")

In [ ]:
#explora os metadados e dados da arquivo importado
qtde_campos = len(dados.domain.attributes)
qtde_cont = sum(1 for a in dados.domain.attributes if a.is_continuous)
qtde_disc = sum(1 for a in dados.domain.attributes if a.is_discrete)
print("%d metadados: %d continuos, %d discretos" % (qtde_campos, qtde_cont, qtde_disc))
print("Nome dos metadados:", ", ".join(dados.domain.attributes[i].name for i in range(qtde_campos)),)
dados.domain.attributes #explora os domínios dos atributos (campos da base de dados)
print("Qtde de Registros:", len(dados)) #explora os dados (quantidade de registros da base de dados)
i = 0 #exibe os primeiros registros para análise dos dados importados
for d in dados[:20]:
  i += 1
  print(i, d)

In [ ]:
#cria arrays das dimensões do problema a ser otimizado
periodo = []
complexidade = [] #1-muito baixa complexidade;2-baixa complexidade;3-média complexidade;4-alta complexidade;e,5-muito alta complexidade
pagina = []
prazo = []
revisao = []
entrega = []
i = 0
for d in dados[:POPULACAO]:
  periodo.append(d[1])
  complexidade.append(d[2])
  pagina.append(d[3])
  prazo.append(d[4])
  revisao.append(d[5])
  entrega.append(d[6])
  print("id:",i,"período:",periodo[i],"complexidade:",complexidade[i],"página:",pagina[i],"prazo:",prazo[i],"revisões:",revisao[i],"entrega:",entrega[i])
  i += 1

In [ ]:
#função custo ou objetivo ou aptidão ou otimização ou fitness - usada para buscar o melhor ponto dentro de um espaço de buscao (melhor global) sem ficar preso em um melhor local
def fitness(problema, posicoes, alfa=0, beta=0):
  total = 0.0
  if problema == 'rosenbrock':
    for i in range(DIMENSOES-1):
      total += 100*(posicoes[i+1] - posicoes[i]**2)**2 + (1-posicoes[i])**2
  elif problema == 'esfera':
    for i in range(DIMENSOES):
      total += posicoes[i]**2
  elif problema == 'custo':
    for i in range(DIMENSOES-1):
      total += 1 / abs(sum([coord ** 2 for coord in posicao]))
  elif problema == 'rota':
    denominador = sum([(caminho.feromonio)**alfa * (1 / caminho.comprimento)**beta for caminho in posicoes])
    distribuicao_probabilidades = None
    if denominador == 0:
      distribuicao_probabilidades = [1 / len(posicoes)  for _ in posicoes]
    else:
      distribuicao_probabilidades = [((caminho.feromonio)**alfa * (1 / caminho.comprimento)**beta) / denominador for caminho in posicoes]
    total = choice(posicoes, 1, p=distribuicao_probabilidades)[0]
  else:
    print('Problema não encontrado!')
  return total

In [ ]:
#particula - unidade base da otimização, posicionada numa determinada posição no espaço de busca do problema, representando uma solução em potencial para o problema
class Particula:
  def __init__(self):
    self.lista_posicao = []
    self.lista_velocidade = []
    self.fitness = np.inf
    self.fitness_pbest = np.inf
    self.lista_posicao_pbest = []

In [ ]:
#inicializa população (particulas)
for i in range(POPULACAO):
  cpx = complexidade[i]
  pag = pagina[i]
  p = Particula()
  for j in range(DIMENSOES):
    if (j == 0):
      posicao = cpx
    elif (j == 1):
      posicao = pag
    else:
      print('Dimensão não encontrado!')
    p.lista_posicao.append(posicao)
    p.lista_velocidade.append(random.uniform(VM[0], VM[1]))
  PARTICULAS.append(p)
for i in range(POPULACAO):
  print("i:",i)
  print(list(PARTICULAS[i].lista_posicao))
  print(list(PARTICULAS[i].lista_posicao_pbest))

In [ ]:
#posiciona as partículas no espaço de busca
print("Plano Cartesiano")
plt.axis([E1,E2,E3,E4])
plt.plot(0,0, marker='*', markersize=15, color='b')
for i in range (POPULACAO):
  particula = PARTICULAS[i]
  d1,d2 = zip(particula.lista_posicao)
  plt.plot(d1,d2, marker='o')
plt.show()

In [ ]:
#calcula o fitness
def calculo_fitness(particula):
  particula.fitness = fitness(FCUSTO, particula.lista_posicao)
  if (particula.fitness < particula.fitness_pbest):
    particula.fitness_pbest = particula.fitness
    particula.lista_posicao_pbest = list(particula.lista_posicao)

In [ ]:
#calcula a velocidade
def atualizacao_velocidade_global(particula, lista_gbest):
  for i in range(DIMENSOES):
    ele1 = random.random()
    ele2 = random.random()
    velocidade_cognitiva = FI*ele1* (particula.lista_posicao_pbest[i] - particula.lista_posicao[i])
    velocidade_social = FS*ele2* (lista_gbest[i] - particula.lista_posicao[i])
    v = CI * particula.lista_velocidade[i] + velocidade_cognitiva + velocidade_social
    if v > LIMITES[1]:
      v = LIMITES[1]
    elif v < LIMITES[0]:
      v = LIMITES[0]
    particula.lista_velocidade[i] = v

In [ ]:
#atualiza a posição da partícula
def atualiza_posicao(particula, bound):
  for i in range(DIMENSOES):
    novo_valor = particula.lista_posicao[i] + particula.lista_velocidade[i]
    if novo_valor > bound[1]:
      novo_valor =  bound[1]
    if novo_valor < bound[0]:
      novo_valor = bound[0]
    particula.lista_posicao[i] = novo_valor

In [ ]:
#otimiza as posições
fitness_gbest = float('inf')
lista_posicao_gbest = []
lista_melhores_valores = []
for i in range(ITERACOES):
  print("Iteração: {:.0f}".format(i+1))
  for j in range(POPULACAO):
    calculo_fitness(PARTICULAS[j])
    if  PARTICULAS[j].fitness < fitness_gbest:
     fitness_gbest = PARTICULAS[j].fitness
     lista_posicao_gbest = list(PARTICULAS[j].lista_posicao)
  for j in range(POPULACAO):
    atualizacao_velocidade_global(PARTICULAS[j],lista_posicao_gbest)
    atualiza_posicao(PARTICULAS[j],LIMITES)
  lista_melhores_valores.append(fitness_gbest)
  plt.axis([E1,E2,E3,E4])
  plt.plot(0,0, marker='*', markersize=10, color='b')
  for i in range(POPULACAO):
    p = PARTICULAS[i]
    d1,d2 = zip(p.lista_posicao)
    plt.plot(d1,d2, marker='o')
  plt.plot(fitness_gbest, marker='x', markersize=15, color='r') #ponto gbest
  plt.show()
  print('Melhor Ponto:',fitness_gbest)
  print("")

In [ ]:
#exibe a curva de convergência das partículas
x = []
y = []
for i in range(len(lista_melhores_valores)):
  x.append(i)
  y.append(lista_melhores_valores[i])
plt.plot(x, y)
plt.title("Curva de Convergência do PSO")
plt.xlabel("Iterações")
plt.ylabel("Tempo")
plt.tight_layout()

In [ ]:
#exibe das posições e otimizações calculadas
posicao_inicial = []
posicao_otimizada = []
for i in range(POPULACAO):
  posicao_inicial.append(PARTICULAS[i].lista_posicao)
  posicao_otimizada.append(PARTICULAS[i].lista_posicao_pbest)
  print("i:",i,"posição:",posicao_inicial[i],"otimização:",posicao_otimizada[i])
#y = []
#ỹ = []
#for i in range(POPULACAO):
#  y.append(int(PARTICULAS[i].lista_posicao[0]))
#  ỹ.append(int(PARTICULAS[i].lista_posicao_pbest[0]))
#print('A:', metrics.accuracy_score(y,ỹ))
#print('P:', metrics.precision_score(y,ỹ,average='macro'))
#print('R:', metrics.recall_score(y,ỹ,average='macro'))
#print('F:', metrics.f1_score(y,ỹ,average='macro'))